##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pesos esparsos usando pruning estrutural

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

O pruning estrutural dos pesos do seu modelo para deixá-los esparsos em um padrão específico pode acelerar o tempo de inferência do modelo com suportes de HW apropriados.

Este tutorial mostra a você como:

- Definir e treinar um modelo com o dataset MNIST usando uma esparsidade estrutural específica
- Converter o modelo após o pruning ao formato tflite
- Visualizar a estrutura dos pesos após o pruning

Para uma visão geral da técnica de pruning para a otimização de modelos, veja a [visão geral do pruning](https://www.tensorflow.org/model_optimization/guide/pruning). Para um tutorial sobre o pruning de pesos em geral, confira [Pruning no Keras](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras).

## Pruning estrutural de pesos

O pruning estrutural zera sistematicamente os pesos do modelo no início do processo de treinamento. Você aplica essas técnicas de pruning a blocos regulares de pesos para acelerar a inferência nos HWs compatíveis, por exemplo: agrupa os pesos no modelo em blocos de quatro e zera dois desses pesos em cada bloco, o que é conhecido como redução *2 por 4*. Essa técnica só se aplica à última dimensão do tensor do peso para o modelo que é convertido pelo TensorFlow Lite. Por exemplo, os pesos da camada `Conv2D` no TensorFlow Lite têm a estrutura `[channel_out, height, width, channel_in]`, e os pesos da camada `Dense` têm a estrutura `[channel_out, channel_in]`. O padrão de esparsidade é aplicado aos pesos na última dimensão: `channel_in`.

Comparada à esparsidade aleatória, a esparsidade estruturada geralmente tem menor exatidão devido à estrutura restritiva. No entanto, ela pode reduzir o tempo de inferência significativamente no hardware compatível.

O pruning pode ser aplicado a um modelo junto com outras técnicas de compressão do modelo para uma melhor taxa de compressão. Veja exemplos de quantização e clustering em [técnicas de otimização colaborativa](https://blog.tensorflow.org/2021/10/Collaborative-Optimizations.html) para mais detalhes.

## Configuração

Prepare o ambiente e os dados de desenvolvimento.

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization
! pip install -q matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

## Baixe e normalize os dados de imagem do dataset [MNIST](https://www.tensorflow.org/datasets/catalog/mnist)

In [ ]:
# Load MNIST dataset.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

## Defina os parâmetros de pruning estrutural

Determine parâmetros para o pruning e especifique o tipo de pruning estrutural. Defina os parâmetros para pruning como `(2, 4)`. Essas configurações significam que, em um bloco de quatro elementos, pelo menos os dois com a menor magnitude são definidos como zero.

Você não precisa configurar o parâmetro `pruning_schedule`. Por padrão, a máscara de pruning é definida como o primeiro passo e não é atualizada durante o treinamento.

In [ ]:
pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

Defina os parâmetros para o pruning aleatório com a esparsidade alvo de 50%.

In [ ]:
pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}

Estabeleça a arquitetura do modelo e especifique quais camadas passarão pelo pruning. O pruning estrutural é aplicado com base nas camadas do modelo que você selecionar.

No exemplo abaixo, só fazemos o pruning de algumas camadas, a segunda `Conv2D` e a primeira `Dense`.

Observe que a primeira camada `Conv2D` não pode passar pelo pruning estrutural. Para isso, ela precisa ter mais de um canal de entrada. Em vez disso, fazemos o pruning aleatório da primeira camada `Conv2D`.

In [ ]:
model = keras.Sequential([
    prune_low_magnitude(
        keras.layers.Conv2D(
            32, 5, padding='same', activation='relu',
            input_shape=(28, 28, 1),
            name="pruning_sparsity_0_5"),
        **pruning_params_sparsity_0_5),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    prune_low_magnitude(
        keras.layers.Conv2D(
            64, 5, padding='same',
            name="structural_pruning"),
        **pruning_params_2_by_4),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    keras.layers.Flatten(),
    prune_low_magnitude(
        keras.layers.Dense(
            1024, activation='relu',
            name="structural_pruning_dense"),
        **pruning_params_2_by_4),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Treine e avalie o modelo.

In [ ]:
batch_size = 128
epochs = 2

model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    callbacks=tfmot.sparsity.keras.UpdatePruningStep(),
    validation_split=0.1)

_, pruned_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Pruned test accuracy:', pruned_model_accuracy)

Remova o wrapper de pruning para que não seja incluído no modelo durante a conversão para o formato do TensorFlow Lite.

In [ ]:
model = tfmot.sparsity.keras.strip_pruning(model)

## Converta o modelo para o formato tflite

In [ ]:
import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

## Visualize e verifique os pesos

Agora visualize a estrutura dos pesos na camada `Dense` após o pruning com a esparsidade 2 por 4. Extraia os pesos do arquivo tflite.

In [ ]:
# Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

Para verificar se selecionamos a camada correta com o pruning, imprima o formato do tensor de peso.

In [ ]:
print(f"Shape of Dense layer is {tensor_data.shape}")

Agora visualizamos a estrutura para um pequeno subconjunto do tensor de peso. A estrutura do tensor de peso é esparsa na última dimensão, usando o padrão `(2,4)`: dois dos quatro elementos são zeros. Para deixar a visualização mais clara, substituímos todos os valores diferentes de zero por 1.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24

subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)

Defina a função auxiliar para desenhar linhas de separação e ver a estrutura claramente.

In [ ]:
def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')

Agora visualize o subconjunto do tensor de peso.

In [ ]:
plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

Visualize os pesos da camada `Conv2D`. A esparsidade estrutural é aplicada no último canal, semelhante à camada `Dense`. Apenas a segunda camada `Conv2D` passa pelo pruning estrutural, conforme indicado acima.

In [ ]:
# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
tensor_name = 'structural_pruning/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[1]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

Semelhante aos pesos da camada `Dense`, a última dimensão do kernel tem uma estrutura (2, 4).

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()

Vamos ver como esses pesos ficam após o pruning aleatório. Depois de extrai-los, exibimos um subconjunto do tensor de peso.

In [ ]:
# Get weights of the convolutional layer that has been pruned with random pruning.
tensor_name = 'pruning_sparsity_0_5/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[0]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tensor_data.shape[0],tf.reduce_prod(tensor_data.shape[1:])])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 0, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Unstructed pruned weights for Conv2D layer")
plt.show()

O Kit de ferramentas para otimização de modelos do TensorFlow inclui um script Python que pode ser usado para verificar quais camadas no modelo do arquivo tflite específico têm os pesos com o pruning estrutural: [`check_sparsity_m_by_n.py`](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py). O comando a seguir demonstra como usar essa ferramenta para verificar a esparsidade 2 por 4 em um determinado modelo.

In [ ]:
! python3 ./tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py --model_tflite=pruned_model.tflite --m_by_n=2,4
